In [ ]:
import requests as rq
import json
import matplotlib.pyplot as plt
from matplotlib import image as mpimg

BASE_URL = 'http://solaris.micc.unifi.it/andreaderosa'

TARGET_TEXT="woman with long hair and crossed hand by leonardo da vinci"
USER_TEXT="with a mountain as background"
DIRECTORY=""

payload = {'prompt':USER_TEXT, 'directory':DIRECTORY, 'target_text':TARGET_TEXT}
response = rq.get(BASE_URL, params=payload)

json_values=response.json()

rq_input=json_values['image']

plt.imshow(rq_input)
plt.axis("off")

In [ ]:
import requests
url = "http://solaris.micc.unifi.it/andreaderosa"
page = requests.get(url)
print(page.status_code)

#200 ok, 404 errore

In [ ]:
import os
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

from flask import Flask
from flask import request
import json

app=Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def handle_request():

  rq_prompt=str(request.args.get('prompt'))
  rq_directory=str(request.args.get('directory'))
  rq_target_text=str(request.args.get('target_text'))

  TARGET_TEXT = rq_target_text
  OUTPUT_DIR = rq_directory
  prompt = rq_prompt

  model_path = OUTPUT_DIR

  scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)
  pipe = StableDiffusionPipeline.from_pretrained(model_path, scheduler=scheduler, torch_dtype=torch.float16).to("cuda")
  target_embeddings = torch.load(os.path.join(model_path, "target_embeddings.pt")).to("cuda")
  optimized_embeddings = torch.load(os.path.join(model_path, "optimized_embeddings.pt")).to("cuda")
  g_cuda = None

  g_cuda = torch.Generator(device='cuda')
  seed = -1
  g_cuda.manual_seed(seed)

  num_samples = 1
  guidance_scale = 5
  num_inference_steps = 100
  height = 512
  width = 512

  pipe.safety_checker = None
  pipe.requires_safety_checker = False

  with autocast("cuda"), torch.inference_mode():
      images = pipe(
          TARGET_TEXT+" "+prompt,
          height=height,
          width=width,
          num_images_per_prompt=num_samples,
          num_inference_steps=num_inference_steps,
          guidance_scale=guidance_scale,
          generator=g_cuda
      ).images

  data_set={'image': images[0]}
  json_dump=json.dumps(data_set)

  return json_dump